In [2]:
import pandas as pd
from pathlib import Path

In [3]:
ruta_crudos = Path("../datos/crudos")
archivos = sorted(ruta_crudos.glob("StreamingHistory_music_*.json"))
archivos

[PosixPath('../datos/crudos/StreamingHistory_music_0.json'),
 PosixPath('../datos/crudos/StreamingHistory_music_1.json')]

In [4]:
lista_df = []

for archivo in archivos:
    print("Cargando:", archivo.name)
    df_temp = pd.read_json(archivo)
    lista_df.append(df_temp)

df = pd.concat(lista_df, ignore_index=True)
df.head()

Cargando: StreamingHistory_music_0.json
Cargando: StreamingHistory_music_1.json


,endTime,artistName,trackName,msPlayed
0,2024-11-23 04:13,Billie Eilish,WILDFLOWER,65598
1,2024-11-26 15:05,Daddy Yankee,Vuelve,265564
2,2024-11-27 04:08,Don Toliver,No Pole,187613
3,2024-11-27 04:12,The Weeknd,Timeless (feat. Playboi Carti),256000
4,2024-11-27 04:12,Travis Scott,FE!N (feat. Playboi Carti),12780


In [5]:
df = df.rename(columns={
    "endTime": "fecha_reproduccion",
    "artistName": "artista",
    "trackName": "cancion",
    "msPlayed": "ms_reproducidos"
})

df.head()
df.columns

Index(['fecha_reproduccion', 'artista', 'cancion', 'ms_reproducidos'], dtype='object')

In [6]:
df["fecha_reproduccion"] = pd.to_datetime(df["fecha_reproduccion"])

# Minutos reproducidos
df["minutos_reproducidos"] = df["ms_reproducidos"] / 60000

f = df["fecha_reproduccion"]

df["anio"] = f.dt.year
df["mes"] = f.dt.month
df["dia"] = f.dt.day
df["dia_semana"] = f.dt.day_name()   # en inglés, igual que Elías
df["hora"] = f.dt.hour

df.head()

,fecha_reproduccion,artista,cancion,ms_reproducidos,minutos_reproducidos,anio,mes,dia,dia_semana,hora
0,2024-11-23 04:13:00,Billie Eilish,WILDFLOWER,65598,1.093300,2024,11,23,Saturday,4
1,2024-11-26 15:05:00,Daddy Yankee,Vuelve,265564,4.426067,2024,11,26,Tuesday,15
2,2024-11-27 04:08:00,Don Toliver,No Pole,187613,3.126883,2024,11,27,Wednesday,4
3,2024-11-27 04:12:00,The Weeknd,Timeless (feat. Playboi Carti),256000,4.266667,2024,11,27,Wednesday,4
4,2024-11-27 04:12:00,Travis Scott,FE!N (feat. Playboi Carti),12780,0.213000,2024,11,27,Wednesday,4


In [7]:
df["usuario"] = "elie"
df[["fecha_reproduccion", "artista", "cancion", "minutos_reproducidos", "usuario"]].head()

,fecha_reproduccion,artista,cancion,minutos_reproducidos,usuario
0,2024-11-23 04:13:00,Billie Eilish,WILDFLOWER,1.093300,elie
1,2024-11-26 15:05:00,Daddy Yankee,Vuelve,4.426067,elie
2,2024-11-27 04:08:00,Don Toliver,No Pole,3.126883,elie
3,2024-11-27 04:12:00,The Weeknd,Timeless (feat. Playboi Carti),4.266667,elie
4,2024-11-27 04:12:00,Travis Scott,FE!N (feat. Playboi Carti),0.213000,elie


In [9]:
from pathlib import Path

ruta_salida = Path("../datos/procesados/elie_limpio.csv")
df.to_csv(ruta_salida, index=False)

ruta_salida

PosixPath('../datos/procesados/elie_limpio.csv')

In [10]:
df_elie_limpio = pd.read_csv(ruta_salida, parse_dates=["fecha_reproduccion"])
df_elie_limpio.head()
df_elie_limpio.shape

(17880, 11)

In [11]:
import sys
from pathlib import Path
import pandas as pd

ruta_raiz = Path("..").resolve()
if str(ruta_raiz) not in sys.path:
    sys.path.append(str(ruta_raiz))

from src.analitica_spotify.consultas import top_artistas

In [12]:
df_elie = pd.read_csv("../datos/procesados/elie_limpio.csv", parse_dates=["fecha_reproduccion"])
top_artistas(df_elie, n=10)

artista
Bad Bunny           2333.956433
Billie Eilish       1574.885700
Drake               1284.598683
The Weeknd          1163.198333
Ludovico Einaudi    1031.841367
Jesse & Joy         1016.144700
Hans Zimmer         1014.437217
Frank Ocean          901.755600
Adele                824.820117
Fairuz               761.744533
Name: minutos_reproducidos, dtype: float64

In [13]:
df_elie = df
df_elie.columns

Index(['fecha_reproduccion', 'artista', 'cancion', 'ms_reproducidos',
       'minutos_reproducidos', 'anio', 'mes', 'dia', 'dia_semana', 'hora',
       'usuario'],
      dtype='object')